## Tutorial - 0: Continuous Forward Schrodinger

Welcome to our first tutorial on Physics-Informed Neural Networks (PINNs) using `PyTorch Lightning` and our new package, **`pinnstorch`**! This guide aims to introduce you to the integration of deep learning and scientific computing via PINNs, which embed physical laws into neural network training.

#### Overview:

- **Mesh Generation and Sampling:** Understanding how to create and use meshes for training PINNs, crucial for defining the domain of our problem.
- **Neural Network Model with PINNs:** How to build and structure a neural network for physical law integration.
- **Physical Laws in Neural Networks:** Detailing the implementation of differential equations within the network using `pde_fn` and `output_fn`.
- **Training and Validation:** Utilizing PyTorch Lightning’s Trainer to train our model, including defining the training data, initial conditions, and handling boundary conditions.
- **Results Visualization:** Saving and analyzing the outcomes of your PINN models.

#### Install Libraries

In [ ]:
!pip install pinnstorch
!pip install lightning

#### Import Libraries

In [ ]:
from typing import Dict

import torch
import numpy as np
import lightning.pytorch as pl

import pinnstorch

### Define Mesh

Physics-Informed Neural Networks (PINNs) require a discretized domain (mesh) over which the physical equations are solved. In `pinnstorch`, there are primarily two approaches to define this mesh:


- **Defining Time and Spatial Domains Separately:** Here, we use `pinnstorch.data.TimeDomain` and `pinnstorch.data.Interval` for creating 1-D spatial domains. These domains are then used to define a `pinnstorch.data.Mesh`.

- **Defining Point Clouds:** This method involves directly utilizing spatio-temporal data (e.g., from experiments or other simulations) to create a mesh using `pinnstorch.data.PointCloud`.

Both approaches should ideally yield the same results.

#### Option 1: Defining Mesh with Separate Time and Spatial Domains

We start by defining a function to read and preprocess the solution data from a file. The output should be a dictionary.

In [ ]:
def read_data_fn(root_path):
    """Read and preprocess data from the specified root path.

    :param root_path: The root directory containing the data.
    :return: Processed data will be used in Mesh class.
    """

    data = pinnstorch.utils.load_data(root_path, "NLS.mat")
    exact = data["uu"]
    exact_u = np.real(exact) # N x T
    exact_v = np.imag(exact) # N x T
    exact_h = np.sqrt(exact_u**2 + exact_v**2) # N x T
    return {"u": exact_u, "v": exact_v, "h": exact_h}

Now, define the time and spatial domains for mesh generation. The choice of these parameters depends on the specific problem being solved and should be set accordingly.

In [ ]:
time_domain = pinnstorch.data.TimeDomain(t_interval=[0, 1.57079633], t_points = 201)
spatial_domain = pinnstorch.data.Interval(x_interval= [-5, 4.9609375], shape = [256, 1])

The mesh is then defined using the time and spatial domains along with the read_data_fn function. 

In [ ]:
mesh = pinnstorch.data.Mesh(root_dir='../data',
                            read_data_fn=read_data_fn,
                            spatial_domain = spatial_domain,
                            time_domain = time_domain)

#### Option 2: Using Point Clouds

For scenarios where the spatial and temporal data along with solutions are directly available (e.g., from experimental measurements), we can create a mesh using `pinnstorch.data.PointCloud`.

**Note:** It's crucial to format the spatial and temporal domain dimensions correctly. Specifically:

- **Spatial Domain:** Each axis should be structured in the shape of $(N \times 1)$, where $N$ represents the number of spatial points.
- **Time Domain:** This should be formatted as $(T \times 1)$, where $T$ indicates the number of time steps.
- **Solution Data:** Each solution variable (e.g., temperature, velocity) should be in the shape of $(N \times T)$, aligning with the spatial and temporal points.

In [ ]:
def read_data_fn(root_path):
    """Read and preprocess data from the specified root path.

    :param root_path: The root directory containing the data.
    :return: Processed data will be used in PointCloud class.
    """

    data = pinnstorch.utils.load_data(root_path, "NLS.mat")

    x = data["x"].T  # N x 1
    t = data["tt"].T  # T x 1
    
    exact = data["uu"]
    exact_u = np.real(exact) # N x T
    exact_v = np.imag(exact) # N x T
    exact_h = np.sqrt(exact_u**2 + exact_v**2) # N x T
    
    return pinnstorch.data.PointCloudData(
            spatial=[x], time=[t], solution={"u": exact_u, "v": exact_v, "h": exact_h}
    )

Now, mesh can be initalize with using the function and the directory to the folder of data.

In [ ]:
mesh = pinnstorch.data.PointCloud(root_dir='./data',
                                  read_data_fn=read_data_fn)

### Define Train datasets

For solving Schrodinger PDE, we have:
- Initial condition
- Periodic boundary condition
- Collection points for the PDE.

#### Initial Condition

Let's start with initial condition of the Schrodinger.
$$ u(0, x) = 2 \text{sech}(x) $$
$$ v(0, x) = 0 $$

For defining initial condition, again we have two options.

- **Sample from the data.**
- **Defining a function for calculating initial condition.**

##### Set number of samples

In [ ]:
N0 = 50

##### Option 1: Sample from the data

In [ ]:
in_c = pinnstorch.data.InitialCondition(mesh = mesh,
                                        num_sample = N0,
                                        solution = ['u', 'v'])

##### Option 2: Defining a function for calculating initial condition

The input of `initial_fun` should be the same as spatial domain of the problem.

In [ ]:
def initial_fun(x):
    return {'u': 2*1/np.cosh(x), 'v': np.zeros_like(x)}

In [ ]:
in_c = pinnstorch.data.InitialCondition(mesh = mesh,
                                        num_sample = N0,
                                        initial_fun = initial_fun,
                                        solution = ['u', 'v'])

The `solution` attribute in `pinnstorch.data.InitialCondition` specifies the solutions (`u` and `v` in our case) to be sampled for initial conditions.

#### Periodic Boundary Condition

The `pinnstorch.data.PeriodicBoundaryCondition` is used to sample periodic points from the upper and lower bounds of the spatial domain (mesh). The `derivative_order` parameter specifies the order of the derivative to be matched at these boundaries. In our case, for the Schrödinger equation, both the function and its first spatial derivative should match at the boundaries, hence `derivative_order = 1`.


$$ u(t,-5) = u(t, 5), $$
$$ v(t,-5) = v(t, 5), $$ 
$$ u_x(t,-5) = u_x(t, 5),$$
$$ v_x(t,-5) = v_x(t, 5) $$

In [ ]:
N_b = 50
pe_b = pinnstorch.data.PeriodicBoundaryCondition(mesh = mesh,
                                                 num_sample = 50,
                                                 derivative_order = 1,
                                                 solution = ['u', 'v'])

#### Mesh Sampler for collection points and solutions

In our problem, the partial differential equations (PDEs) governing the dynamics are given by:

$$ f_u := u_t + 0.5v_{xx} + v(u^2 +v^2),$$
$$ f_v := v_t + 0.5u_{xx} + u(u^2 +v^2) $$

To find the solutions to these PDEs using a neural network, we must sample points from the domain at which the network will be trained to satisfy these equations. This sampling process is crucial for training our PINN. We utilize the `pinnstorch.data.MeshSampler` for this purpose, specifying the following:

- **Number of Sample Points (N_f):** We choose to sample 20,000 points from the domain. This number is a balance between computational efficiency and the need for a sufficiently dense sampling to capture the dynamics of the PDEs.
- **Mesh (mesh):** This parameter defines the spatial-temporal domain from which the points will be sampled.
- **Collection Points:** We define `['f_u', 'f_v']` as the targets for our collection points. These are not direct outputs from the neural network but are derived from the network outputs and their derivatives (We will define `pde_fn` function later). The PINN will be trained such that these expressions tend towards zero, aligning with the PDE constraints.

Here's the code to implement this sampler:

In [ ]:
N_f = 20000
me_s = pinnstorch.data.MeshSampler(mesh = mesh,
                                   num_sample = N_f,
                                   collection_points = ['f_v', 'f_u'])

### Define Validation dataset

For validation, we sample all points from the mesh to evaluate our model comprehensively. Model will be validated for solutions of `u`, `v`, and `h`.

**Note:** If `num_sample` is not specified, the sampler will use the entire mesh for data sampling.

In [ ]:
val_s = pinnstorch.data.MeshSampler(mesh = mesh,
                                    solution = ['u', 'v', 'h'])

### Define Neural Networks

Here, we try to define a neural network for solving the problem. For defining a neural network, we should set number of layers and the name of the outputs. Also, domain bounds should be defined. The `lb` and `ub` parameters represent the lower and upper bounds of the spatial-temporal domain, helping in normalizing inputs to the network. Therefore, the inputs of this network are `x` and `t`, and the outputs of this network are `u` and `v`.

In [ ]:
net = pinnstorch.models.FCN(layers = [2, 100, 100, 100, 100, 2],
                            output_names = ['u', 'v'],
                            lb=mesh.lb,
                            ub=mesh.ub)

### Define `pde_fn` and `output_fn` functions

Now, we define `pde_fn` and `output_fn`.
- **`output_fn`:** is applied to the network's output, adding any necessary post-processing computations. For example, in our case, `h(x,t) = u(x,t)**2 + v(x,t)**2`, thus, we define this equation in `output_fn`.
- **`pde_fn`:** formulates the PDE constraints, which will be used by the `MeshSampler` to compute the loss at the collection points. 

#### `output_fn` function

**Note:** `output_fn` should always have these inputs:
- **Outputs:** It is output of the network. In our case, this dictionary should have two output: `u` and `v`.
- **Spatial domains:** These are the spatial domain variables. In our case, because our problem has 1-D spatial domain, the input just have `x`. For example, if we had 2-D space, we need another input for that dimention. For example, the inputs from `(outputs, x, t)` will be changed to `(outputs, x, y, t)`.
- **Time domin:** The last input of `output_fn` function always should be time.

In [ ]:
def output_fn(outputs: Dict[str, torch.Tensor],
              x: torch.Tensor,
              t: torch.Tensor):
    """Define `output_fn` function that will be applied to outputs of net."""

    outputs["h"] = torch.sqrt(outputs["u"] ** 2 + outputs["v"] ** 2)

    return outputs

#### `pde_fn` function

The inputs are similar to `output_fn`. Only if we have extra variables for training (i.g. in inverse problems), we should add input at the end of inputs. For example, `(outputs, x, t)` will be `(outputs, x, t, extra_variable)`. `extra_variable` is always a dictionary.

In [ ]:
def pde_fn(outputs: Dict[str, torch.Tensor],
           x: torch.Tensor,
           t: torch.Tensor):   
    """Define the partial differential equations (PDEs)."""
    u_x, u_t = pinnstorch.utils.gradient(outputs["u"], [x, t])
    v_x, v_t = pinnstorch.utils.gradient(outputs["v"], [x, t])

    u_xx = pinnstorch.utils.gradient(u_x, x)[0]
    v_xx = pinnstorch.utils.gradient(v_x, x)[0]

    outputs["f_u"] = u_t + 0.5 * v_xx + (outputs["u"] ** 2 + outputs["v"] ** 2) * outputs["v"]
    outputs["f_v"] = v_t - 0.5 * u_xx - (outputs["u"] ** 2 + outputs["v"] ** 2) * outputs["u"]

    return outputs

### Define PINNDataModule and PINNModule

To integrate with Lightning, we utilize two specialized modules:

- `PINNDataModule` (inherited from `LightningDataModule`) manages data.
- `PINNModule` (derived from `LightningModule`) handles the model, compilation, and various enhancements like AMP.

#### Define `PINNDataModule`
Here, we define collection points, initial condition, and preiodic boundary condition as training datasets, and also, we set validation set. `PINNDataModule` is used for defining training, validation, prediction, and test datasets.

In [ ]:
train_datasets = [me_s, in_c, pe_b]
val_dataset = val_s
datamodule = pinnstorch.data.PINNDataModule(train_datasets = [me_s, in_c, pe_b],
                                            val_dataset = val_dataset,
                                            pred_dataset = val_s)

#### Define `PINNModule`

`PINNModule` handle several things. Here, we will explore the inputs of this class:
- **net:**  The neural network.
- **pde_fn:** The function representing the PDE to solve.
- **optimizer:**  (Optional) The optimizer for training. The default is Adam
- **loss_fn:** (Optional) The loss function to use, either "sse" or "mse". The default is "sse".
- **scheduler:** (Optional) Learning rate scheduler. The default is None.
- **scaler:** (Optional) Gradient scaler for AMP. The default is `torch.cuda.amp.GradScaler`.
- **extra_variables:** (Optional) Extra variables in inverse problems. The default is None.
- **output_fn:** (Optional) function to process the model's output. The default is None.
- **runge_kutta:** (Optional) Runge-Kutta method for solving PDEs in discrete mode. The default is None.
- **cudagraph_compile:** Flag to enable CUDA Graph compilation. It works only with a single GPU. The default is True.
- **jit_compile:** (Optional) Flag to enable JIT compilation. The default is True.
- **amp:** (Optional) Flag to enable Automatic Mixed Precision (AMP). The default is False.
- **inline:** (Optional) Flag to enable inline mode in JIT compilation. The default is False.

In this example, we initalize `PINNModule` with defined variables. We set Adam optimizer and try to compile the model with CUDA Graph. The loss function here is Mean Square Error (MSE).

In [ ]:
model = pinnstorch.models.PINNModule(net = net,
                                     pde_fn = pde_fn,
                                     output_fn = output_fn,
                                     loss_fn = 'mse')

### Setting Up the Trainer

For training our model, we configure a trainer in PyTorch Lightning. Currently, our setup uses a single GPU, as models compiled with CUDA Graph don't support multiple GPUs yet. For a comprehensive understanding of the trainer's options and functionalities, refer to the [official documentation](https://lightning.ai/docs/pytorch/stable/api/lightning.pytorch.trainer.trainer.Trainer.html). For example, arguments that you can set are:
- **accelerator:**  Supports passing different accelerator types such as `cpu`, `gpu`, `tpu`, `ipu`, `hpu`, `mps`, and `auto`.
- **devices:** The devices to use. Can be set to a positive number (int or str), a sequence of device indices (list or str), the value -1 to indicate all available devices should be used, or "auto" for automatic selection based on the chosen accelerator. Default: "auto".
- **max_epochs:** Stop training once this number of epochs is reached.
- **max_steps:** Stop training after this number of steps.
- ...

In our example, we configure the trainer for CPU use, specifying one device:

In [ ]:
trainer = pl.Trainer(accelerator='cpu', devices=1)

### Training


In [ ]:
trainer.fit(model=model, datamodule=datamodule)

### Validation

In [ ]:
trainer.validate(model=model, datamodule=datamodule)

### Plotting

For plotting, we need predict the results, and then, we should concatenate the results.

In [ ]:
preds_list = trainer.predict(model=model, datamodule=datamodule)
preds_dict = pinnstorch.utils.fix_predictions(preds_list)

In [ ]:
pinnstorch.utils.plot_schrodinger(mesh=mesh,
                                  preds=preds_dict,
                                  train_datasets=train_datasets,
                                  val_dataset=val_dataset,
                                  file_name='out')